In [1]:
import os

import findspark
findspark.init("/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3")
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


# warehouse_location points to the default location for managed databases and tables
os.environ["PYSPARK_PYTHON"]="/home/pc/TestJupyter/opt/spark-3.3.0/venv-spark/bin/python39"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.0  pyspark-shell'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", "hdfs://10.123.51.78:8000/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive")\
    .config("hive.metastore.uris", "thrift://g2.bigtop.it:9083")\
    .enableHiveSupport() \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d97da1dc-e448-494b-bccd-446a77c0aac7;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 144ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

22/10/03 10:29:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [23]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import desc
import pandas as pd

In [ ]:
## Overall highest tag ngram

In [114]:
df = spark.read.format("avro").load("NO_PUNCT_GRAM.avro")

AnalysisException: Path does not exist: file:/home/pc/Assignment/SocialMedia/Main/NO_PUNCT_GRAM.avro

In [112]:
df

DataFrame[row_id: bigint, sentence_id: int, pos: int, token_gram: string, tag_gram: string, lang_gram: string, gram_type: string]

In [16]:
one = df.rdd.map(lambda x : (x.tag_gram, 1))
mapRslt = one.reduceByKey(lambda K1_V1, K1_V2 : K1_V1+K1_V2).toDF(["tag_gram","frequency"])

In [85]:
descDF =                                                                                                                                                                                  \
    mapRslt                                                                                                                                                                            \
        .select("tag_gram","frequency", F.row_number()
        .over(Window.partitionBy()
                .orderBy(desc(mapRslt['frequency'])))
        .alias("id"))

In [86]:
descDF

DataFrame[tag_gram: string, frequency: bigint, id: int]

In [87]:
p = descDF.limit(15)

In [88]:
frame = p.toPandas()

22/10/03 07:15:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/03 07:15:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/03 07:15:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [89]:
tag_grams = frame["tag_gram"].tolist()

In [90]:
examples = [', '.join(spark.sql(f"select token_gram from socialmediangrams where tag_gram = '{pattern}' limit 5").toPandas()["token_gram"].tolist()) for pattern in tag_grams]

In [91]:
frame.insert(3, "eg",examples, True)

In [92]:
frame

,tag_gram,frequency,id,eg
0,NOUN NOUN,7552078,1,"repeart lg, material kain, bubble wrap, tq sel..."
1,NOUN ADJ,4471212,2,"keadaan baik, order lmbt, barang berpatutan, r..."
2,ADJ NOUN,3822446,3,"cam botol, cam botol, berbaloi treats, berbalo..."
3,NOUN ADV,3188510,4,"first time, first time, order lagi, first time..."
4,ADP NOUN,3108214,5,"dari seller, dari seller, dari seller, dari se..."
5,ADV VERB,2985662,6,"time beli, time beli, time beli, time beli, ti..."
6,VERB NOUN,2941232,7,"beli barang, duk shah, repeat order, boleh rep..."
7,PROPN PROPN,2939942,8,"cotton soo, thank you, recommended seller, kak..."
8,PROPN NOUN,2805492,9,"alam order, lajuu terima, good job, vietnam ba..."
9,ADV ADJ,2361346,10,"sangat berbaloi, memang okay, memang terbaik, ..."


In [4]:
import shutil
shutil.rmtree("temp")

In [ ]:
## Overall highest frequency token ngram

In [101]:
df = spark.sql("select token_gram from socialmediangrams")

In [104]:
one2 = df.rdd.map(lambda x : (x.token_gram, 1))
mapRslt2 = one2.reduceByKey(lambda K1_V1, K1_V2 : K1_V1+K1_V2).toDF(["token_gram","frequency"])

In [105]:
descDF2 =                                                                                                                                                                                  \
    mapRslt2                                                                                                                                                                            \
        .select("token_gram","frequency", F.row_number()
        .over(Window.partitionBy()
                .orderBy(desc(mapRslt2['frequency'])))
        .alias("id"))

In [106]:
p = descDF2.limit(15)

In [107]:
frame = p.toPandas()

22/10/03 07:23:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/03 07:23:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/03 07:23:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [108]:
frame

,token_gram,frequency,id
0,fast delivery,707074,1
1,tq seller,664500,2
2,good condition,652890,3
3,in good,571314,4
4,terbaik terbaik,462016,5
5,thank you,451624,6
6,good product,419842,7
7,terima kasih,398516,8
8,received in,394732,9
9,barang sampai,359026,10
